# Vehicle Routing Problem with Time Windows (VRPTW)

Interactive demonstration of a genetic algorithm solving the VRPTW.

## What is VRPTW?

The Vehicle Routing Problem with Time Windows (VRPTW) is a complex optimization problem where:
- Multiple vehicles must visit customers
- Each customer has a time window for service
- Vehicles have limited capacity
- The goal is to minimize total distance traveled while respecting all constraints

This is an NP-hard problem, so we use a genetic algorithm to find good approximate solutions.

In [ ]:
import sys
import os
sys.path.append(os.path.join('..', 'src'))

import numpy as np
import matplotlib.pyplot as plt
from vrptw import Customer, Vehicle, VRPTWProblem, VRPTWGeneticAlgorithm

# Set random seed for reproducible results
np.random.seed(42)
print("🚛 VRPTW Interactive Demo Loaded!")

## Create a Sample Problem

Let's create a problem with 15 customers and 2 vehicles.

In [ ]:
# Create depot
depot = Customer(id=0, x=0, y=0, demand=0, ready_time=0, due_time=480, service_time=0)

# Create customers with time windows
customers = []
for i in range(1, 16):
    # Position customers in a circle
    angle = (i-1) * 2 * np.pi / 15
    radius = 20 + np.random.uniform(-5, 5)
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    
    # Time windows: morning and afternoon shifts
    if i <= 8:
        ready_time = np.random.uniform(60, 120)  # Morning
        due_time = ready_time + np.random.uniform(30, 60)
    else:
        ready_time = np.random.uniform(180, 240)  # Afternoon  
        due_time = ready_time + np.random.uniform(30, 60)
    
    demand = np.random.uniform(1, 4)
    service_time = np.random.uniform(5, 15)
    
    customer = Customer(
        id=i, x=x, y=y, demand=demand,
        ready_time=ready_time, due_time=due_time, service_time=service_time
    )
    customers.append(customer)

# Create vehicles
vehicles = [
    Vehicle(id=1, capacity=20, depot_return_time=480),
    Vehicle(id=2, capacity=25, depot_return_time=480)
]

# Create problem instance
problem = VRPTWProblem(customers, vehicles, depot)

print(f"✅ Created VRPTW problem:")
print(f"   - {len(customers)} customers")
print(f"   - {len(vehicles)} vehicles (capacities: {[v.capacity for v in vehicles]})")
print(f"   - Total demand: {sum(c.demand for c in customers):.1f} units")

## Visualize the Problem

Let's see what our problem looks like before solving it.

In [ ]:
import plotly.graph_objects as go

# Create problem visualization
fig = go.Figure()

# Plot depot
fig.add_trace(go.Scatter(
    x=[depot.x], y=[depot.y],
    mode='markers+text',
    marker=dict(size=25, color='black', symbol='star'),
    text=['Depot'],
    textposition="top center",
    name='Depot'
))

# Plot customers with time windows
for customer in customers:
    fig.add_trace(go.Scatter(
        x=[customer.x], y=[customer.y],
        mode='markers',
        marker=dict(
            size=12,
            color=customer.ready_time,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Start Time"),
            line=dict(width=2, color='white')
        ),
        text=[f'Customer {customer.id}<br>Demand: {customer.demand:.1f}<br>Time: {customer.ready_time:.0f}-{customer.due_time:.0f}'],
        hoverinfo='text',
        name=f'Customer {customer.id}'
    ))

# Update layout
fig.update_layout(
    title='VRPTW Problem Instance<br>(Colors indicate service start times)',
    xaxis_title='X Coordinate',
    yaxis_title='Y Coordinate',
    width=800,
    height=600,
    showlegend=False
)

fig.show()

## Run the Genetic Algorithm

Now let's solve this problem using our genetic algorithm.

In [ ]:
# Create genetic algorithm solver
ga = VRPTWGeneticAlgorithm(
    problem=problem,
    population_size=50,  # Smaller for demo
    generations=100,
    mutation_rate=0.2,
    crossover_rate=0.8,
    tournament_size=3
)

print("🧬 Running genetic algorithm...")
best_solution, best_routes, best_fitness = ga.evolve()
print(f"✅ Optimization complete! Best fitness: {best_fitness:.2f}")

## Analyze the Solution

Let's examine the routes found by the genetic algorithm.

In [ ]:
# Analyze routes
print("📊 Route Analysis:")
total_distance = 0
total_customers = 0

for i, route in enumerate(best_routes):
    if route.customers:
        total_distance += route.total_distance
        total_customers += len(route.customers)
        load = sum(c.demand for c in route.customers)
        
        print(f"Vehicle {route.vehicle_id}:")
        print(f"  - Customers: {[c.id for c in route.customers]}")
        print(f"  - Distance: {route.total_distance:.2f}")
        print(f"  - Load: {load:.1f}/{problem.vehicles[i].capacity}")
        print(f"  - Total time: {route.total_time:.1f}")
        print(f"  - Feasible: {'✅ Yes' if route.feasible else '❌ No'}")
        print()

print(f"📈 Summary:")
print(f"   - Total distance: {total_distance:.2f}")
print(f"   - Customers served: {total_customers}/{len(customers)}")
print(f"   - Fitness score: {best_fitness:.2f}")

## Visualize the Solution

Now let's create a beautiful visualization of the optimal routes.

In [ ]:
# Create solution visualization
fig_solution = ga.visualize_solution(best_routes)
fig_solution.show()

## Fitness Evolution

Let's see how the genetic algorithm improved over time.

In [ ]:
# Plot fitness evolution
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(ga.best_fitness_history, label='Best Fitness', linewidth=2, color='blue')
ax.plot(ga.avg_fitness_history, label='Average Fitness', linewidth=2, color='red', alpha=0.7)

ax.set_xlabel('Generation')
ax.set_ylabel('Fitness (Total Distance + Penalties)')
ax.set_title('Genetic Algorithm Fitness Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

improvement = (ga.best_fitness_history[0] - ga.best_fitness_history[-1]) / ga.best_fitness_history[0] * 100
print(f"🎯 Total improvement: {improvement:.1f}%")

## Experiment with Parameters

Try modifying the genetic algorithm parameters and see how they affect the solution quality.

### Ideas to explore:
1. Change the population size
2. Adjust mutation and crossover rates
3. Modify the number of generations
4. Try different tournament sizes
5. Create a different problem instance

### Example: Higher mutation rate

In [ ]:
# Experiment with higher mutation rate
ga_experiment = VRPTWGeneticAlgorithm(
    problem=problem,
    population_size=50,
    generations=100,
    mutation_rate=0.4,  # Higher mutation
    crossover_rate=0.8,
    tournament_size=3
)

print("🧬 Running experiment with higher mutation rate...")
best_solution_exp, best_routes_exp, best_fitness_exp = ga_experiment.evolve()
print(f"✅ Experiment complete! Best fitness: {best_fitness_exp:.2f} (vs {best_fitness:.2f} original)")

# Compare results
if best_fitness_exp < best_fitness:
    print("🎉 Higher mutation rate performed better!")
else:
    print("📉 Higher mutation rate performed worse or similar.")

## Conclusion

This notebook demonstrated:

1. **Problem Formulation**: How to define a VRPTW with customers, time windows, and vehicle constraints
2. **Genetic Algorithm**: Implementation of selection, crossover, and mutation operators
3. **Solution Visualization**: Interactive plots showing routes and time windows
4. **Performance Analysis**: Fitness evolution and parameter sensitivity

### Key Takeaways:
- Genetic algorithms can find good solutions to complex optimization problems
- The VRPTW is challenging due to multiple constraints (capacity, time windows)
- Visualization helps understand both the problem and solution quality
- Parameter tuning can significantly impact algorithm performance

### Next Steps:
- Implement more sophisticated crossover operators
- Add local search improvement heuristics
- Compare with other metaheuristics (simulated annealing, ant colony)
- Solve larger problem instances